In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

In [2]:
import joblib
data = joblib.load('nsmc.pkl')
locals().update(data)

In [3]:
import pandas as pd 
nsmc = pd.read_csv('nsmc_train.csv', sep='\t')

In [4]:
# 단어문서행렬은 x
x = dtm

In [5]:
# 2천개의 label 뽑아주기 
y = nsmc.label.values[:2000]

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1984)

In [8]:
# tensorflow 사용하여 로지스틱 회귀분석 모델 정의
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 압축을 풀지않고, 압축된 상태에서 fitting 시키는 방법

In [11]:
type(x_train)
# scipy csr 포멧을 텐서 플로우 포멧(coo방식)으로 맞춰줘야함 

scipy.sparse.csr.csr_matrix

In [12]:
x_coo = x_train.tocoo()

In [13]:
x_coo.row

array([   0,    0,    0, ..., 1599, 1599, 1599])

In [14]:
x_coo.col

array([  1, 119, 140, ..., 754, 951, 952])

In [15]:
x_coo.data

array([1, 1, 1, ..., 1, 1, 7], dtype=int64)

In [16]:
index = np.column_stack((x_coo.row, x_coo.col))

In [17]:
index
# 순서가 섞여있을때는 주의 해야함 

array([[   0,    1],
       [   0,  119],
       [   0,  140],
       ...,
       [1599,  754],
       [1599,  951],
       [1599,  952]])

In [19]:
x_train_sparse = tf.SparseTensor(index, x_coo.data, x_coo.shape)

In [21]:
# 위의 index 행번호나 열번호가 섞여 있을때 모델에 집어넣지 못하기 때문에 순서를 정렬해줌
x_train_sparse = tf.sparse.reorder(x_train_sparse)

In [22]:
# 위의 tensor희소행렬을 만들면 압축을 풀지않은 상태로 학습진행 할수 있음 
model.fit(x_train_sparse, y_train, epochs=1)

50/50 [==============================] - 0s 675us/step - loss: 0.6883 - accuracy: 0.5400


## 데이터가 큰경우에 텐서플로우의 희소행렬로 바꾸어 압축된 상태로 학습진행가능